### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()


groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [ ]:
#!pip install langchain_groq


   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 2/2 [langchain_groq]



In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028A9FA5C280>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028A9FA29720>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
response= model.invoke("What Nietzsche meant by God is dead?")
response.content

'Nietzsche\'s famous proclamation, "God is dead," is a complex and often misinterpreted statement. It\'s not meant to be taken literally as a statement about the physical death of a deity. \n\nHere\'s a breakdown of what Nietzsche meant:\n\n**1. The Decline of Religious Belief:**\n\nNietzsche lived in a time when traditional religious authority was waning in Europe. Enlightenment ideas, scientific discoveries, and social changes were challenging the foundations of Christian morality and belief. Nietzsche saw this decline as a profound shift in human history, a "death" of the old ways of thinking.\n\n**2. The Loss of Meaning and Values:**\n\nFor Nietzsche, God wasn\'t just a figure of faith, but also a source of meaning and moral values. With God\'s "death," traditional sources of morality and purpose were lost. This created a sense of nihilism, a feeling of meaninglessness and emptiness.\n\n**3. The Need for Revaluation:**\n\nNietzsche believed that the "death of God" wasn\'t something

In [ ]:
#!pip install langchain_core

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model=ChatGroq(model="Gemma2-9b-It")

# Pipe operator (|) chains components
chain = prompt | model

response = chain.invoke({"topic": "programming"})
print(response.content)

Why do programmers prefer dark mode? 

Because light attracts bugs! 🐞  😄  


Let me know if you'd like to hear another one! 



In [11]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()  # Converts to clean text
response = chain.invoke({"topic": "AI"})
print(response)

Why did the AI cross the road? 

Because it was programmed to!  🐔🤖😂 


Let me know if you'd like to hear another one!  😄  




## Translation

In [18]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Hindi"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [19]:
result

AIMessage(content='नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 21, 'total_tokens': 43, 'completion_time': 0.04, 'prompt_time': 0.003614227, 'queue_time': 0.245624951, 'total_time': 0.043614227}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec9589ce-2cfb-478c-b107-1bf8e34c908c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 22, 'total_tokens': 43})

In [14]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n'

In [15]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'नमस्ते, आप कैसे हैं?  (Namaste, aap kaise hain?) \n'

## Prompt Templates

In [24]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following in {language}."

prompt = ChatPromptTemplate.from_messages(
        [("system", generic_template),
         ("user"," {text}")
         ]
)

In [25]:
result=prompt.invoke({"language":"Hindi","text":"Hello"})

In [30]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following in Hindi.', additional_kwargs={}, response_metadata={}), HumanMessage(content=' Hello', additional_kwargs={}, response_metadata={})])

In [29]:
result.to_messages()

[SystemMessage(content='Translate the following in Hindi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' Hello', additional_kwargs={}, response_metadata={})]

In [35]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Hindi","text":"Hello"})

'नमस्ते (Namaste) \n'